In [ ]:
"""
SpaceX Falcon 9 - EDA with SQL
"""

import sqlite3
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('spacex_clean.csv')
print(f"📊 Data loaded: {len(df)} records\n")

# Create SQLite database
conn = sqlite3.connect('spacex.db')
df.to_sql('launches', conn, if_exists='replace', index=False)
print("✅ SQLite database created\n")

print("="*70)
print("SPACEX FALCON 9 - SQL ANALYSIS RESULTS")
print("="*70)

# ============================================
# QUERY 1: Unique launch site names
# ============================================
print("\n1. UNIQUE LAUNCH SITE NAMES:")
print("-" * 70)
query1 = pd.read_sql("""
    SELECT DISTINCT LaunchSiteName
    FROM launches
    WHERE LaunchSiteName IS NOT NULL
    ORDER BY LaunchSiteName
""", conn)
print(query1)
print(f"Total unique sites: {len(query1)}")

# ============================================
# QUERY 2: Launch sites beginning with 'CCA'
# ============================================
print("\n\n2. LAUNCH SITES BEGINNING WITH 'CCA' (First 5 records):")
print("-" * 70)
query2 = pd.read_sql("""
    SELECT FlightNumber, Date, LaunchSiteName, Success
    FROM launches
    WHERE LaunchSiteName LIKE 'CCA%'
    LIMIT 5
""", conn)
print(query2)

# ============================================
# QUERY 3: Total payload mass by NASA
# ============================================
print("\n\n3. TOTAL PAYLOAD MASS CARRIED FOR NASA:")
print("-" * 70)
query3 = pd.read_sql("""
    SELECT COALESCE(SUM(PayloadMass), 0) as TotalPayload
    FROM launches
    WHERE Customer = 'NASA'
""", conn)
total_payload = query3['TotalPayload'].values[0]
print(f"Total payload: {total_payload:,.2f} kg")

# ============================================
# QUERY 4: Average payload mass for Falcon 9 v1.1 - FIXED
# ============================================
print("\n\n4. AVERAGE PAYLOAD MASS FOR FALCON 9 v1.1:")
print("-" * 70)
query4 = pd.read_sql("""
    SELECT AVG(PayloadMass) as AvgPayload, COUNT(*) as Count
    FROM launches
    WHERE BoosterVersion LIKE '%v1.1%' OR RocketName LIKE '%v1.1%'
""", conn)

if query4['AvgPayload'].values[0] is not None:
    avg_payload = query4['AvgPayload'].values[0]
    count = query4['Count'].values[0]
    print(f"Average payload: {avg_payload:,.2f} kg")
    print(f"Number of launches: {count}")
else:
    print("No data found for Falcon 9 v1.1")

# ============================================
# QUERY 5: First successful ground landing date
# ============================================
print("\n\n5. FIRST SUCCESSFUL GROUND PAD LANDING DATE:")
print("-" * 70)
query5 = pd.read_sql("""
    SELECT MIN(Date) as FirstLanding, FlightNumber, LaunchSiteName
    FROM launches
    WHERE LandingOutcome = 'Success'
    AND (LandingType LIKE '%ground%' OR LandingType LIKE '%RTLS%')
    GROUP BY FlightNumber, LaunchSiteName
    ORDER BY Date
    LIMIT 1
""", conn)
print(query5)

# ============================================
# QUERY 6: Successful drone ship landings (payload 4000-6000)
# ============================================
print("\n\n6. BOOSTERS WITH SUCCESSFUL DRONE SHIP LANDING (Payload 4000-6000 kg):")
print("-" * 70)
query6 = pd.read_sql("""
    SELECT DISTINCT BoosterVersion, COUNT(*) as Count
    FROM launches
    WHERE LandingOutcome = 'Success'
    AND (LandingType LIKE '%drone%' OR LandingType LIKE '%ASDS%')
    AND PayloadMass BETWEEN 4000 AND 6000
    GROUP BY BoosterVersion
""", conn)
print(query6)

# ============================================
# QUERY 7: Total successful and failed missions
# ============================================
print("\n\n7. TOTAL SUCCESSFUL VS FAILED MISSION OUTCOMES:")
print("-" * 70)
query7 = pd.read_sql("""
    SELECT
        CASE WHEN Success = 1 THEN 'Success' ELSE 'Failure' END as Outcome,
        COUNT(*) as Count,
        ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM launches), 2) as Percentage
    FROM launches
    GROUP BY Success
""", conn)
print(query7)

# ============================================
# QUERY 8: Booster versions with maximum payload
# ============================================
print("\n\n8. BOOSTER VERSIONS THAT CARRIED MAXIMUM PAYLOAD:")
print("-" * 70)
query8 = pd.read_sql("""
    SELECT BoosterVersion, PayloadMass as MaxPayload, FlightNumber, Date
    FROM launches
    WHERE PayloadMass = (SELECT MAX(PayloadMass) FROM launches)
    LIMIT 5
""", conn)
print(query8)

# ============================================
# QUERY 9: Failed drone ship landings in 2015
# ============================================
print("\n\n9. FAILED DRONE SHIP LANDINGS IN 2015:")
print("-" * 70)
query9 = pd.read_sql("""
    SELECT FlightNumber, Date, BoosterVersion, LaunchSiteName, LandingOutcome
    FROM launches
    WHERE Year = 2015
    AND LandingOutcome = 'Failure'
    AND (LandingType LIKE '%drone%' OR LandingType LIKE '%ASDS%')
""", conn)
if len(query9) > 0:
    print(query9)
else:
    print("No failed drone ship landings found in 2015")

# ============================================
# QUERY 10: Rank landing outcomes (2010-06-04 to 2017-03-20)
# ============================================
print("\n\n10. RANKING OF LANDING OUTCOMES (2010-06-04 to 2017-03-20):")
print("-" * 70)
query10 = pd.read_sql("""
    SELECT
        COALESCE(LandingOutcome, 'No attempt') as LandingOutcome,
        COUNT(*) as Count
    FROM launches
    WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
    GROUP BY LandingOutcome
    ORDER BY Count DESC
""", conn)
print(query10)

# Close connection
conn.close()

print("\n" + "="*70)
print("🎉 ALL 10 SQL QUERIES COMPLETED SUCCESSFULLY!")
print("="*70)


📊 Data loaded: 205 records

✅ SQLite database created

SPACEX FALCON 9 - SQL ANALYSIS RESULTS

1. UNIQUE LAUNCH SITE NAMES:
----------------------------------------------------------------------
    LaunchSiteName
0     CCSFS SLC 40
1       KSC LC 39A
2  Kwajalein Atoll
3      VAFB SLC 4E
Total unique sites: 4


2. LAUNCH SITES BEGINNING WITH 'CCA' (First 5 records):
----------------------------------------------------------------------
Empty DataFrame
Columns: [FlightNumber, Date, LaunchSiteName, Success]
Index: []


3. TOTAL PAYLOAD MASS CARRIED FOR NASA:
----------------------------------------------------------------------
Total payload: 9,813.00 kg


4. AVERAGE PAYLOAD MASS FOR FALCON 9 v1.1:
----------------------------------------------------------------------
No data found for Falcon 9 v1.1


5. FIRST SUCCESSFUL GROUND PAD LANDING DATE:
----------------------------------------------------------------------
                FirstLanding  FlightNumber LaunchSiteName
0  2015-12-22 